In [ ]:
# 다운로드가 필요한 모듈, 라이브러리
# pip install lightgbm
# pip install catboost

SyntaxError: invalid syntax (2543105895.py, line 2)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier


In [73]:
# 1) CSV 불러오기 & match_id 생성
data = pd.read_csv('Matches.csv', parse_dates=['MatchDate'])
data = data.reset_index().rename(columns={'index':'match_id'})

# 2) 홈/원정 각각 long 포맷으로 전환
home = data[['match_id','MatchDate','HomeTeam','FTHome','FTAway']].copy()
home = home.assign(
    team           = home['HomeTeam'],
    goals_for      = home['FTHome'],
    goals_against  = home['FTAway'],
    venue          = 'Home'
)[['match_id','MatchDate','team','goals_for','goals_against','venue']]

away = data[['match_id','MatchDate','AwayTeam','FTAway','FTHome']].copy()
away = away.assign(
    team           = away['AwayTeam'],
    goals_for      = away['FTAway'],
    goals_against  = away['FTHome'],
    venue          = 'Away'
)[['match_id','MatchDate','team','goals_for','goals_against','venue']]

matches_long = pd.concat([home, away], ignore_index=True)

# 3) 정렬하고 인덱스 재설정 (꼭 필요)
matches_long = matches_long.sort_values(['team','MatchDate']).reset_index(drop=True)

# 4) 과거 3·5경기 득실 합계 계산 (transform 이용)
for N in (3, 5):
    # 먼저 “현재 경기” 제외를 위해 shift()
    shifted_gf = matches_long.groupby('team')['goals_for']     .shift()
    shifted_ga = matches_long.groupby('team')['goals_against'] .shift()

    # rolling 합계 계산
    matches_long[f'GF{N}'] = (shifted_gf
                              .groupby(matches_long['team'])
                              .transform(lambda x: x.rolling(N).sum()))
    matches_long[f'GA{N}'] = (shifted_ga
                              .groupby(matches_long['team'])
                              .transform(lambda x: x.rolling(N).sum()))

# 5) 홈/Away별로 다시 뽑아서 이름 바꾸기
home_stats = (
    matches_long[matches_long['venue']=='Home']
    .set_index('match_id')[['GF3','GA3','GF5','GA5']]
    .rename(columns={
        'GF3':'GF3Home','GA3':'GA3Home',
        'GF5':'GF5Home','GA5':'GA5Home'
    })
)
away_stats = (
    matches_long[matches_long['venue']=='Away']
    .set_index('match_id')[['GF3','GA3','GF5','GA5']]
    .rename(columns={
        'GF3':'GF3Away','GA3':'GA3Away',
        'GF5':'GF5Away','GA5':'GA5Away'
    })
)

# 6) map으로 원본 data에 컬럼 추가
for col in home_stats.columns:
    data[col] = data['match_id'].map(home_stats[col])
for col in away_stats.columns:
    data[col] = data['match_id'].map(away_stats[col])

# 7) 불필요해진 match_id 제거 (선택)
data = data.drop(columns=['match_id'])

# 8) 결과 확인
data.info()

C:\Users\User\AppData\Local\Temp\ipykernel_3356\845658055.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Matches.csv', parse_dates=['MatchDate'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230557 entries, 0 to 230556
Data columns (total 56 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Division     230557 non-null  object        
 1   MatchDate    230557 non-null  datetime64[ns]
 2   MatchTime    99072 non-null   object        
 3   HomeTeam     230557 non-null  object        
 4   AwayTeam     230557 non-null  object        
 5   HomeElo      141597 non-null  float64       
 6   AwayElo      141528 non-null  float64       
 7   Form3Home    229057 non-null  float64       
 8   Form5Home    229057 non-null  float64       
 9   Form3Away    229057 non-null  float64       
 10  Form5Away    229057 non-null  float64       
 11  FTHome       230554 non-null  float64       
 12  FTAway       230554 non-null  float64       
 13  FTResult     230554 non-null  object        
 14  HTHome       175977 non-null  float64       
 15  HTAway       175977 non-null  floa

In [74]:
# 2. 데이터 전처리
# 2-1 데이터 전처리 :데이터 연도 기준 필터링
# 프리미어 리그(epl)의 2022~2023연도 데이터를 사용
data = data[ data['Division'] == 'E0']   # 프리미어 리그(epl) 데이터 추출

# 2-2 데이터 전처리 : 학습에 사용할 column만 추출
# 'MatchTime' 임시 제거 
columns = ['MatchDate', 'HomeTeam', 'AwayTeam', 'HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'OddHome', 'OddDraw', 'OddAway', 'FTResult', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize', 'HandiHome', 'HandiAway', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away'
]
data = data[columns]

In [75]:
# 2-3 데이터 전처리 : 결측치 확인
data.isnull().sum()

# 결측치 있는 행 제거
# data = data.dropna(subset=['HomeElo', 'AwayElo']).reset_index(drop=True)

data = data.dropna(subset=columns).copy()

In [76]:
# 2-4 데이터 전처리 : Standardize(표준화), OneHotEncoding(원-핫 인코딩)
log_columns = ['HomeElo', 'AwayElo', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiHome', 'HandiAway'] # 로그 변환할 columns
standarize_columns = ['HomeElo', 'AwayElo', 'OddHome', 'OddDraw', 'OddAway', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'HandiSize', 'HandiHome', 'HandiAway', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away']  # 표준화할 columns
encoding_columns = ['HomeTeam', 'AwayTeam']  # 원-핫 인코딩할 columns

# 분산이 큰 배당률 관련 columns와 Elo columns는 표준화 전에 log scale을 먼저 적용
data[log_columns] = np.log1p(data[log_columns])

# Pipeline에서 전처리 해줄 ColumnsTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('standardize', StandardScaler(), standarize_columns),  # 표준화
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True), encoding_columns)  # 원-핫 인코딩
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)


In [77]:
data['MatchDate'] = pd.to_datetime(data['MatchDate'])  # MatchDate를 datetime 형식으로 변환

data_train = data[(data['MatchDate'] < '2025-01-01')]   # 학습 데이터 : 24/25 시즌제외 모든 데이터 추출
data_test = data[(data['MatchDate'] >= '2025-01-01')]   # 테스트 데이터 : 24/25 시즌 데이터 추출

# # MatchDate 제거
data_train = data_train.drop(columns=['MatchDate']) 
data_test = data_test.drop(columns=['MatchDate'])  

#  featuer, target 분리

X_train = data_train.drop(columns=['FTResult'])  # Feature columns
X_test = data_test.drop(columns=['FTResult'])  # Feature columns
y_train = data_train['FTResult'].map({'H':0, 'D':1, 'A':2})  # Target column
y_test = data_test['FTResult'].map({'H':0, 'D':1, 'A':2})  # Target column

In [78]:
# 학습, 테스트 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y,
#     test_size=0.2,           # 20 % 검증(또는 0.25 등)
#     random_state=42,         
#     stratify=y               # 클래스 비율 유지 
# )

In [92]:
# lighstgbm 파이프 라인 정의
pipe_lightgbm = Pipeline(
    steps=[
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', LGBMClassifier(
            objective='multiclass',  # 다중 클래스 분류
            num_class=3,  # 홈 승, 무승부, 원정 승
            n_estimators   = 1400,
            learning_rate  = 0.035,
            max_depth      = -1,          # 자동
            num_leaves     = 63,          # 2^(max_depth) -1 근사
            min_data_in_leaf = 25,        # 최소 leaf 노드 데이터 수
            colsample_bytree = 0.8,
            subsample        = 0.8,
            reg_alpha        = 0.1,
            reg_lambda       = 1.0,
            random_state     = 42,
            class_weight     = 'balanced'
        ))  # LightGBM 
    ]
)

# catboost 파이프 라인 정의
pipe_catboost = Pipeline(
    steps = [
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', CatBoostClassifier(
            loss_function='MultiClass', # 다중 클래스
            iterations = 1200,          # 트리 개수
            learning_rate = 0.05,       # 학습률
            depth = 6,                  # 트리 깊이
            l2_leaf_reg = 3,            # L2 정규화
            random_seed = 42,           
        ))  # CatBoost
    ]
)


pipe_logistic = Pipeline(
    steps=[
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', LogisticRegression(
            max_iter = 1000,              # 최대 반복 횟수
            multi_class = 'multinomial',  # 다중 클래스 분류
            class_weight= 'balanced',     # 클래스 불균형 처리
            solver= 'lbfgs',              # 최적화 알고리즘
        ))  # 로지스틱 회귀
    ]

)

In [ ]:
# stacking 모델 정의
stack = StackingClassifier(
    estimators = [
        ('lightgbm', pipe_lightgbm),
        ('catboost', pipe_catboost),
        ('logistic', pipe_logistic)
    ],
    final_estimator= LogisticRegression(max_iter=1000, multi_class='multinomial'),
    stack_method = 'predict_proba',  # 확률 예측을 위한 stacking
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # 교차 검증 설정
    n_jobs= -1  # 모든 CPU 코어 사용
)

In [ ]:
# 모델 학습
# pipe_lightgbm.fit(X_train, y_train)
stack.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3539
[LightGBM] [Info] Number of data points in the train set: 7396, number of used features: 107
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


c:\project\module_project\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['HomeElo', 'AwayElo',
                                                   'OddHome', 'OddDraw',
                                                   'OddAway', 'Form3Home',
                                                   'Form5Home', 'Form3Away',
                                                   'Form5Away', 'HandiSize',
                                                   'HandiHome', 'HandiAway',
                                                   'GF3Home', 'GA3Home',
                                                   'GF5Home', 'GA5Home',
                                                   'GF3Away', 'GA3Away',
                                                   'GF5Away', 'GA5Away']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['HomeTeam', 'AwayTeam'])])),
                ('classifier',
                 LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
                                learning_rate=0.035, min_data_in_leaf=25,
                                n_estimators=1400, num_class=3, num_leaves=95,
                                objective='multiclass', random_state=42,
                                reg_alpha=0.1, reg_lambda=1.0,
                                subsample=0.8))])

In [ ]:
# 예측
# y_pred_lightgbm  = pipe_lightgbm.predict(X_test)
# y_prob_lightgbm  = pipe_lightgbm.predict_proba(X_test)   # shape = (n_samples, 3)

y_pred = stack.predict(X_test)
y_prob = stack.predict_proba(X_test)


[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25


c:\project\module_project\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\project\module_project\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print("Accuracy :", accuracy_score(y_test, y_pred))
print("\nClassification Report\n", classification_report(y_test, y_pred))

# (선택) 로그-로스 — 다중 클래스 확률 평가
print("Log-loss :", log_loss(y_test, y_prob))

# (선택) 혼동 행렬
print("Confusion Matrix\n", confusion_matrix(y_test, y_pred))

Accuracy : 0.48677248677248675

Classification Report
               precision    recall  f1-score   support

           0       0.51      0.72      0.60        78
           1       0.18      0.15      0.16        40
           2       0.65      0.42      0.51        71

    accuracy                           0.49       189
   macro avg       0.45      0.43      0.42       189
weighted avg       0.49      0.49      0.47       189

Log-loss : 1.2299110214386577
Confusion Matrix
 [[56 15  7]
 [25  6  9]
 [28 13 30]]
